In [ ]:
##分别进行直接聚类和降噪后聚类两种方法
import matplotlib.image as mpimg
plt.rcParams['font.size'] = 8
plt.rcParams['figure.figsize'] = (8, 8)
from mpl_toolkits import mplot3d
from mpl_toolkits.mplot3d import Axes3D
import os
import numpy as np
import cv2
import pickle
from PIL import Image
from collections import Counter
import matplotlib.image as imgplt
from sklearn.decomposition import PCA
from matplotlib.animation import FuncAnimation
from keras.models import Sequential, load_model
from keras.layers import Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.utils import image_utils
from keras import backend as K
from sklearn.cluster import KMeans
import time
import pandas as pd
from keras.optimizers import Adam
import tensorflow
import torch
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.metrics import adjusted_rand_score
from sklearn.metrics import normalized_mutual_info_score



# 获取图片大小和维度
img = cv2.imread("/home/GaoBX/StudyRGB/1.png")
height, width, channels = img.shape
print('图片大小（宽 x 高）：', width, 'x', height)
print('图片维度（通道数）：', channels)
imgplot = plt.imshow(img)
plt.show()



##导入图片

img_width, img_height, channels = 224, 224, 3
input_shape = (img_width, img_height, channels)

def load_data():
    dir = '/home/GaoBX/StudyRGB/'
    files = ["%s/%s" % (dir, x) for x in sorted(os.listdir(dir), key=lambda x: int(x.split(".")[0]))]
    arr = np.empty((len(files), img_width, img_height, channels), dtype=np.float32)
    for i, imgfile in enumerate(files):
        img = image_utils.load_img(imgfile, target_size=(224, 224))
        x = image_utils.img_to_array(img).reshape(img_width, img_height, channels)
        x = x.astype('float32') / 255.
        arr[i] = x
    return arr

X = load_data()
print(X.shape)


##读取细胞类别文件，一般为“meatdata”文件
df = pd.read_csv("/home/GaoBX/work/metadata.csv",index_col=0)
true_labels = df['cluster']


##直接聚类

X = np.array(X).reshape(11227,-1)

n_clusters = 6
kmeans = KMeans(n_clusters=6)
kmeans.fit(X)
labels = kmeans.labels_
cluster_centers = kmeans.cluster_centers_

ari = adjusted_rand_score(true_labels, labels)
nmi = normalized_mutual_info_score(true_labels, labels)
print(f"ARI score: {ari}")
print(f"NMI score: {nmi}")




##降噪后聚类

import tensorflow as tf

tf.config.list_physical_devices('GPU')

X = torch.tensor(X, dtype=torch.float32).permute(0, 3, 1, 2)  # 调整张量维度以匹配PyTorch要求
print(X.shape)


from torchvision.models import resnet50

# 检查是否有可用的GPU，并将模型和数据移动到GPU上
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

import torch
import torch.nn as nn


class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()

        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding='same'),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 16, kernel_size=3, padding='same'),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(16, 8, kernel_size=3, padding='same'),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )

        # Decoder
        self.decoder = nn.Sequential(
            nn.Conv2d(8, 8, kernel_size=3, padding='same'),
            nn.ReLU(),
            nn.Upsample(scale_factor=2, mode='nearest'),
            nn.Conv2d(8, 16, kernel_size=3, padding='same'),
            nn.ReLU(),
            nn.Upsample(scale_factor=2, mode='nearest'),
            nn.Conv2d(16, 32, kernel_size=3, padding='same'),
            nn.ReLU(),
            nn.Upsample(scale_factor=2, mode='nearest'),
            nn.Conv2d(32, 3, kernel_size=3, padding='same'),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x
    

##AutoClass based on ResNet50

import torch
import torch.nn as nn
from torchvision.models import resnet50


class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()

        # Encoder: Use pre-trained ResNet50 without fully connected layers
        resnet = resnet50(pretrained=True)
        self.encoder = nn.Sequential(*list(resnet.children())[:-2])

        # Decoder
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(2048, 1024, kernel_size=2, stride=2, padding=0, output_padding=0),
            nn.ReLU(),
            nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2, padding=0, output_padding=0),
            nn.ReLU(),
            nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2, padding=0, output_padding=0),
            nn.ReLU(),
            nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2, padding=0, output_padding=0),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2, padding=0, output_padding=0),
            nn.ReLU(),
            nn.Conv2d(64, 3, kernel_size=3, padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


import torch
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

# 创建Autoencoder模型实例
model = Autoencoder().to(device)
print(model)

# 定义损失函数和优化器
loss_function = nn.BCELoss()
optimizer = optim.Adadelta(model.parameters())

# 加载数据（请使用您的实际数据替换X）
dataset = TensorDataset(X, X)
data_loader = DataLoader(dataset, batch_size=128, shuffle=True)

# 训练模型
epochs = 50
for epoch in range(epochs):
    for batch_idx, (data, target) in enumerate(data_loader):
        # 将数据和目标移动到相同的设备（GPU或CPU）
        data, target = data.to(device), target.to(device)

        # 前向传播
        output = model(data)

        # 计算损失
        loss = loss_function(output, target)

        # 反向传播
        optimizer.zero_grad()
        loss.backward()

        # 更新权重
        optimizer.step()

        # 打印训练进度
        if batch_idx % 10 == 0:
            print(f"Epoch [{epoch + 1}/{epochs}], Step [{batch_idx}/{len(data_loader)}], Loss: {loss.item()}")

# 保存训练好的模型
torch.save(model.state_dict(), "autoencoder.pth")

# 定义模型
model = Autoencoder()
model.load_state_dict(torch.load("./autoencoder.pth", map_location=device))

# 将数据输入模型并得到降维后的结果

result = model(X)
result = result.detach().numpy()

result
X = X.cpu().numpy()

X_encoded_reshape = result.reshape(result.shape[0], result.shape[1] * result.shape[2] * result.shape[3])
print(X_encoded_reshape.shape)

n_clusters = 6
kmeans = KMeans(n_clusters=6)
kmeans.fit(X_encoded_reshape)
labels = kmeans.labels_
cluster_centers = kmeans.cluster_centers_

ari = adjusted_rand_score(true_labels, labels)
nmi = normalized_mutual_info_score(true_labels, labels)
print(f"ARI score: {ari}")
print(f"NMI score: {nmi}")



##聚类结果二维可视化
from sklearn.manifold import TSNE 

tsne = TSNE(n_components=2, random_state=42, n_jobs=-1)
##tsne_coords = tsne.fit_transform(X)
tsne_coords = tsne.fit_transform(X_encoded_reshape)

colors = {
    0: 'maroon',
    1: 'darkgreen',
    2: 'darkblue',
    3: 'teal',
    4: 'magenta',
    5: 'sienna',
    6: 'olive'
}
fig = plt.figure()
for i in range(len(tsne_coords)):
    plt.scatter(tsne_coords[i, 0], tsne_coords[i, 1], color=colors[labels[i]])

legend_elements = [plt.Line2D([0], [0], marker='o', color='w', label=f'Cluster {i}', markerfacecolor=color, markersize=10) for i, color in colors.items()]
legend = plt.legend(handles=legend_elements, loc='center left', bbox_to_anchor=(1, 0.5))

fig.add_artist(legend)
plt.show()


##聚类结果三维可视化

pca = PCA(n_components=3)
#pca.fit(X)
#total_photo_3d = pca.transform(X)
pca.fit(X_encoded_reshape)
total_photo_3d = pca.transform(X_encoded_reshape)
centers = labels

fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111, projection='3d')
markers = ['o', 'o', 'o', 'o', 'o', 'o', 'o']

for i in range(28):
    ax.scatter(total_photo_3d[labels == i, 0], 
               total_photo_3d[labels == i, 1], 
               total_photo_3d[labels == i, 2],
               marker = markers[i],
               label=f'Cluster {i+1}')
    
    ax.tick_params(axis='x', which='major', labelsize=12, length=4, width=1)
    ax.tick_params(axis='y', which='major', labelsize=12, length=4, width=1)
    ax.tick_params(axis='z', which='major', labelsize=12, length=4, width=1)
    
    ax.view_init(elev=0, azim=90)
    
    #ax.set_xticklabels('') 
    #ax.set_yticklabels('')
    #ax.set_zticklabels('')
    
fig.set_size_inches(10, 7)
plt.show()
